The Knowledge Graph populated in this code will be used for filtering/matching assets based on the scene description they provide. Thus, the asset ontology will not necessarily contain complete information about an asset.

In [5]:
!pip install rdflib
!pip install SPARQLWrapper
!pip install owlready2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
from rdflib import Graph, URIRef, Literal, BNode
from rdflib.namespace import FOAF, XMLNS, XSD, RDF, RDFS, OWL

g = Graph()
g.parse("megascan_assets_ontology.ttl")

print(len(g))


123


In [13]:
from rdflib import Namespace
import json
import re

environments_count = 0


# Open asset file
f = open('/content/tk4tcfhfa.json')

data = json.load(f) # json file loaded as dictionary
n = Namespace("http://www.semanticweb.org/szm/megascan-assets-ontology#")

# For the code segment below, apply loop on all asset files in directory


# Fixed of-interest properties for searching asset file
size_property = URIRef(str(n)+"Property_Size")
g.add((size_property, RDF.type, n.Property))

age_property = URIRef(str(n)+"Property_Age")
g.add((age_property, RDF.type, n.Property))

orientation_property = URIRef(str(n)+"Property_Orientation")
g.add((orientation_property, RDF.type, n.Property))

color_property = URIRef(str(n)+"Property_Color")
g.add((color_property, RDF.type, n.Property))


# Create asset and set asset name
name = data["name"]     # obtain name of asset, convert it to RDF format literal
name_processed = re.sub("[^a-zA-Z0-9]", "_", name)
print(name)
print(name_processed)
asset = URIRef(str(n)+"Asset_"+name_processed)  # create a resource in the Assets Knowledge Graph (KG)
g.add((asset, RDF.type, n.Asset))
g.add((asset, n.assetName, Literal(name)))     # Add triple to KG: {new Asset resource, nameRelation (from schema), name obtained from asset file}

try:
  # Set asset ID
  id = Literal(str(data["id"]))     # obtain name of asset, convert it to RDF format literal
  g.add((asset, n.assetID, id))
except:
  print("No asset ID specified for asset", asset)

try:
  # Set asset search tags
  tags = data["tags"]
  print("number of tags = ", len(tags))
  for t in tags:
    tag = Literal(str(t))
    g.add((asset, n.assetTag, tag))
except:
  print("No search tags specified for asset", asset)

try:
  # Create category individuals and set their names, then assign asset categories
  categories = data["categories"]
  print("number of categories = ", len(categories))
  for c in categories:
    category = URIRef(str(n)+"Category_"+str(re.sub("[^a-zA-Z0-9]", "_", c)))    # these categories should be unique in ontology
    g.add((category, RDF.type, n.Category))
    category_label = Literal(c)
    g.add((category, RDFS.label, category_label))
    g.add((asset, n.assetBelongsToCategory, category))
except:
  print("No categories specified for asset", asset)
# Create property individuals of size, age, orientation and color
# Other properties are not needed for filtering assets
try:
  properties = data["properties"]
  print("number of properties = ", len(properties))
  for key,value in properties.items():
    if key == "size":
      print(value, Literal(value))
      g.add((asset, n.assetHasProperty, size_property))
      g.add((asset, n.assetSizeProperty, Literal(value)))
    elif key == "age":
      print(value, Literal(value))
      g.add((asset, n.assetHasProperty, age_property))
      g.add((asset, n.assetAgeProperty, Literal(value)))
    elif key == "orientation":
      print(value, Literal(value))
      g.add((asset, n.assetHasProperty, orientation_property))
      g.add((asset, n.assetOrientationProperty, Literal(value)))
    elif key == "color":
      print(value, Literal(value))
      g.add((asset, n.assetHasProperty, color_property))
      g.add((asset, n.assetColorProperty, Literal(value)))
except:
  print("No properties specified for asset", asset)


unique_biomes = set()
unique_regions = set()

# Create region and biome individuals, then link to assets
try:
  biome = URIRef(str(n)+"Biome_"+data["environment"]["biome"])
  region = URIRef(str(n)+"Region_"+data["environment"]["region"])
  if biome not in unique_biomes or region not in unique_regions:
    environments_count +=1
    environment = URIRef(str(n)+"Environment_"+str(environments_count))
    g.add((asset, n.assetHasEnvironment, environment))

    g.add((biome, RDFS.label, Literal(data["environment"]["biome"])))
    g.add((environment, n.environmentHasBiome, biome))

    g.add((region, RDFS.label, Literal(data["environment"]["region"])))
    g.add((environment, n.environmentHasRegion, region))

  unique_biomes.add(biome)
  unique_regions.add(region)
  
except:
  print("No environment with biome and region specified for asset", asset)


print(len(g))

f.close()

g.serialize(destination='Populated_Assets_KG.ttl', format='turtle')


Mossy Mounds
Mossy_Mounds
number of tags =  14
number of categories =  3
number of properties =  2
No properties specified for asset http://www.semanticweb.org/szm/megascan-assets-ontology#Asset_Mossy_Mounds
158


<Graph identifier=N444739b696f94873a493053fe5b144d0 (<class 'rdflib.graph.Graph'>)>